In [ ]:
# %%time
import pandas as pd
import requests
import numpy as np

# Get NSE Scrip expiry dates
expurl = "https://www.nseindia.com/live_market/dynaContent/live_watch/fomwatchsymbol.jsp?key=NIFTY&Fut_Opt=Futures"

exphtml = requests.get(expurl).content

fnotable = pd.read_html(exphtml, match='Expiry Date', header=0)[0]
fnoexpiry = fnotable['Expiry Date'][0:1]   # Takes only the first date

# Get the list of equity scrips
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
paisa = pd.read_html(paisaurl, header=0)[1]          # It's the second table in the url

# Remove VIX and NIFTY 
symbol = paisa.loc[~(paisa.Symbol.str.contains('VIX') | paisa.Symbol.str.contains('NIFTY'))].Symbol

# Replace & by %26 for NSE
symbol.replace('&', '%26')


# Combine Expiry and Scrips to a new url DataFrame
urldf = pd.concat([symbol] * len(fnoexpiry), keys=fnoexpiry).reset_index().drop('level_1', axis=1)

# NSE URLs
nseurl = "https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?&instrument=OPTSTK&symbol="

# Make a Dataframe of the NSE URLs
urldf['URL'] = nseurl + urldf['Symbol'] + '&date=' + urldf['Expiry Date']
pd.set_option('display.max_colwidth', -1)

# urldf = urldf.head() # Keep only the first 5 rows for brevity

def getchain(u, e, s):
    chainhtml = requests.get(u).content
    chain = pd.read_html(chainhtml)[1][:-1]  # read the first table and drop the total
    chain.columns=chain.columns.droplevel(0) # drop the first row of the header
    chain = chain.drop('Chart', 1)           # drop the charts
    
    try:            # match='Underlying Stock:' gives errors when table is not found
        underlyingtbl = pd.read_html(chainhtml, match='Underlying Stock:')[0][1]
        underlying = underlyingtbl.iloc[0]
        u = float(underlying.split(' ')[3])

        chain['Underlying'] = u                  # price of the underlying
        chain['Expiry'] = e                      # expiry date
        chain['Symbol'] = s                      # symbol

        return chain
    
    except:
        pass

chainheader = ['Call_OI', 'Call_OI_Chng', 'Call_Volume', 'Call_IV', 'Call_LTP', 'Call_NetChng', 'Call_BidQty', \
               'Call_BidPrice', 'Call_AskPrice', 'Call_AskQty', 'Strike', \
               'Put_BidQty', 'Put_BidPrice', 'Put_AskPrice', 'Put_AskQty', 'Put_NetChng', 'Put_LTP', 'Put_IV', \
               'Put_Volume', 'Put_OI_Chng', 'Put_OI', 'Underlying', 'Expiry', 'Symbol']

v = np.vectorize(getchain)
output = pd.concat(v(urldf.URL, urldf['Expiry Date'], urldf.Symbol))
output.columns = chainheader

output = output.replace("-",0)
output = output[(output.Call_LTP != 0) | (output.Put_LTP != 0)].reset_index(drop=True)

In [ ]:
output[['Symbol', 'Call_IV', 'Put_IV', 'Strike Price', 'Underlying', 'Call_LTP', 'Put_LTP']]